# Pydantic

Pydantic là một thư viện mạnh mẽ cho việc xác thực dữ liệu và quản lý thiết lập cấu trúc dữ liệu và đối tượng.

## Python type hint

Về cơ bản, các ngôn ngữ lập trình được chia làm 2 loại là **Statically typed** và **Dynamically typed**
- **Statically typed**: khi lập trình, ta phải khai báo kiểu của biến một cách tường minh, kiểu dữ liệu của chúng phải được biết tại thời điểm biên dịch (compile time)
-  **Dynamically typed**: kiểu của biến được liên kết với giá trị tại thời điểm chạy (run time), không cần khai báo kiểu của một biến khi lập trình, nó sẽ tự động ép kiểu khi nhận được giá trị. 

Python thuộc kiểu ***Dynamically typed***. Mục đích của **type hint** là cung cấp một cú pháp tiêu chuẩn cho chú thích kiểu của dữ liệu đầu vào và đầu ra. Type hint giúp thông báo tới người đọc và IDEs về các kiểu dữ liệu mong đợi. Điều này thì được khuyến khích chứ không bắt buộc.

Và vì nó là **type hint**, nên nếu trong quá trình chạy pipeline mà kiểu dữ liệu không đúng theo định đạng nhưng ko gây ra error thì code vẫn được chấp nhận


In [1]:
def add(a: int, b: int) -> int:
    return a + b


add(1, 2)  # đúng


add(0.2, 1)  # sai kiểu dữ liệu nhưng run code không lỗi --> đúng


add("a", "b")  # sai kiểu dữ liệu nhưng run code không lỗi --> đúng


'ab'

## How to define datatype metadata

### Sử dụng datatype truyền thống của python

In [22]:
def add(
    a: int,
    b: int,
    c: bool,
    d: float,
    e: list,
    f: dict,
    g: set,
    h: list[str],  # each item in list is string
    i: tuple[
        int, str, float
    ],  # a tuple with 3 items, an int, another int, and a str
    k: set[bytes],  #  a set, and each of its items is of type bytes
    l: dict[str, float],  # a dict with key is string and value is float
    m: str | int,  # a value type is string or integer
    n: str | None,  # string or None
) -> int:
    return int(a) + b


### `Field` thêm metadata cho dữ liệu gốc

`Annotated` để thêm phần ghi chú `Field` cho datatype gốc

**`Field`**:

- `default` : default value
- `alias` : đặt biệt danh cho trường thông tin khác với tên trong trường dữ liệu được xác thực
- `validation_alias` : đặt biệt danh cho trường thông tin giống `alias` nhưng chỉ sử dụng only for validation
- `serialization_alias` được sử dụng khi tên trường nội bộ của mô hình giống `alias`, nhưng không nhất thiết phải là tên bạn muốn sử dụng khi tuần tự hóa mô hình
- `strict`: cho phép được sử dụng trong **strict mode** = không được phép ép kiểu ? (xem bảng bảng tham chiếu ep kiểu [table](https://docs.pydantic.dev/latest/concepts/conversion_table/))
- `exclude` : can be used to control which fields should be excluded from the model when exporting the model

Các trường không được tính đến trong định nghĩa mô hình sẽ không được phân tích.

In [26]:
from pydantic import HttpUrl, PastDate
from pydantic import Field
from pydantic import validate_call
from typing import Annotated
from uuid import uuid4


@validate_call(validate_return=True)
def process_payload(
    url: HttpUrl,  # là dạng url hyperlink
    name: Annotated[
        str, Field(min_length=2, max_length=15)
    ],  # string có độ dài ký tự là 2 --> 15
    birth_date: PastDate,  # là date nhưng là giá trị quá khứ
    id: str = Field(
        default_factory=lambda: uuid4().hex
    ),  # sử dụng trực tiếp Field
) -> str:
    return url, name, birth_date, id


payload = {
    "url": "httpss://example.com",
    "name": "A",
    "birth_date": "2024-12-12",
    "id": "100017727",
}
try:
    process_payload(**payload)
except Exception as e:
    print(e)

3 validation errors for process_payload
url
  URL scheme should be 'http' or 'https' [type=url_scheme, input_value='httpss://example.com', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/url_scheme
name
  String should have at least 2 characters [type=string_too_short, input_value='A', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/string_too_short
birth_date
  Date should be in the past [type=date_past, input_value='2024-12-12', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/date_past


#### For Numeric
- `gt` : greater than
- `lt` : less than
- `ge` : greater than or equal to
- `le` : less than or equal to
- `multiple_of` : a multiple of the given number
- `allow_inf_nan` : allow 'inf', '-inf', 'nan' values


In [ ]:
from pydantic import BaseModel, Field


class Foo(BaseModel):
    positive: int = Field(gt=0)
    non_negative: int = Field(ge=0)
    negative: int = Field(lt=0)
    non_positive: int = Field(le=0)
    even: int = Field(multiple_of=2)
    love_for_pydantic: float = Field(allow_inf_nan=True)

#### For Decimal
- `max_digits` : Maximum number of digits within the Decimal. It does not include a zero before the decimal point or trailing decimal zeroes.
- `decimal_places` : Maximum number of decimal places allowed. It does not include trailing decimal zeroes.


In [28]:
from decimal import Decimal
from pydantic import BaseModel, Field


class Foo(BaseModel):
    precise: Decimal = Field(max_digits=5, decimal_places=2)


foo = Foo(precise=Decimal("123.45"))
print(foo)
# > precise=Decimal('123.45')

precise=Decimal('123.45')


#### For String
- `min_length` : Minimum length of the string.
- `max_length` : Maximum length of the string.
- `pattern` : A regular expression that the string must match.

In [27]:
from pydantic import BaseModel, Field


class Foo(BaseModel):
    short: str = Field(min_length=3)
    long: str = Field(max_length=10)
    regex: str = Field(pattern=r"^\d*$")


foo = Foo(short="foo", long="foobarbaz", regex="123")
print(foo)

short='foo' long='foobarbaz' regex='123'


#### For Dataclass

- `init`: Whether the field should be included in the __init__ of the dataclass.
- `init_var`: Whether the field should be seen as an init-only field in the dataclass.
- `kw_only`: Whether the field should be a keyword-only argument in the constructor of the dataclass.

In [ ]:
from pydantic import BaseModel, Field
from pydantic.dataclasses import dataclass


@dataclass
class Foo:
    bar: str
    baz: str = Field(
        init_var=True
    )  # not included in the model_dump() output, since it is an init-only field.
    qux: str = Field(kw_only=True)


class Model(BaseModel):
    foo: Foo


model = Model(foo=Foo("bar", baz="baz", qux="qux"))

#### For Discriminator: phân loại

- `discriminator` : sử dụng khi muốn control giá trị theo 1 attribute từ 1 số class nào đó

In [29]:
from typing import Literal

from pydantic import BaseModel, Field


class Cat(BaseModel):
    pet_type: Literal["cat"]
    age: int


class Dog(BaseModel):
    pet_type: Literal["dog"]
    age: int


class Model(BaseModel):
    # pet chỉ nhật được trong các giá trị là 'cat' hoặc 'dog' là attribute pet_type của các class định nghĩa
    pet: Cat | Dog = Field(discriminator="pet_type")

#### For property class

Sử dụng `property` trong `class` thường được sử dụng khi tính toán 1 trường thông tin từ các trường khác hoặc việc tính toán trở nên phức tạp hoặc tốn nhiều chi phí, do đó các attribute này thường được tính khi call hoặc khi cần.

- `computed_field` áp dụng cho các atrribute trong class được tính toán theo `property` nhưng vẫn đảm bảo sẽ không tính toán các attribute này cho đến khi được call


In [42]:
from pydantic import BaseModel, computed_field


class Box(BaseModel):
    width: float
    height: float
    depth: float

    @computed_field
    @property  # attribute này sẽ không được tính toán cho đến khi call
    def volume(self) -> float:
        print("đã chạy dòng code tính attribute volume")
        return self.width * self.height * self.depth


b = Box(width=1, height=2, depth=3)

`volume` vẫn được validate datatype khi được call

In [43]:
print(b.model_dump())

đã chạy dòng code tính attribute volume
{'width': 1.0, 'height': 2.0, 'depth': 3.0, 'volume': 6.0}


## Type of validation

Có 3 cách phổ biến để sử dụng được validate:
- Nếu valicate áp dụng cho function: `validate_call`
- Nếu validate áp dụng cho class: kế thừa `BaseModel` (nếu cần validate cả datatype + datavalue) hoặc dùng decorator `pydantic.dataclasses.dataclass` (nếu chỉ cần validate đơn giản và ưu tiên performance)

### Validate hàm - `validate_call`

`validate_call` để xác thực các lần gọi hàm dựa trên type hint và chú thích (annotations)
- Pydantic được sử dụng để đảm bảo dữ liệu bạn truyền vào mỗi lần gọi hàm tuân theo các kiểu dữ liệu mong đợi (type hints).
- Pydantic sẽ cố gắng hiểu ý của bạn và sử dụng ép kiểu (type coercion). Điều này có thể dẫn đến việc các giá trị string được truyền vào lần gọi hàm bị chuyển đổi ngầm thành kiểu dữ liệu mong đợi.
    > Ép kiểu không phải lúc nào cũng khả thi, và trong trường hợp đó, `ValidationError` sẽ được phát sinh.

In [11]:
import pydantic


@pydantic.validate_call
def add(a: int, b: int) -> int:
    return a + b


add(2, 3)

5

báo lỗi khi sai kiểu dữ liệu input

In [ ]:
add("a", 3)

> ValidationError: 1 validation error for add 0
> 
>   Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='a', input_type=str]
>    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing

 sẽ đúng nếu ép kiểu thành công

In [6]:
add("3", "3")

6

Lỗi nếu ép kiểu từ float sang int mà giá trị bị thay đổi:
- `3.0` --> int (**OKE**)
- `0.3` --> int (**bị thay đổi**)

In [ ]:
add(3.0, 0.3)

>Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=0.3, input_type=float]
>    For further information visit https://errors.pydantic.dev/2.7/v/int_from_float

`validate_return=True` nếu muốn validate giá trị trả về, (mặc định là `False`), phướng pháp ép kiểu cũng được áp dụng

> ví dụ output của hàm định nghĩa là int, tuy nhiên output thực tế là string dạng int thì vẫn OKE

In [18]:
from pydantic import validate_call


@validate_call(validate_return=True)
def add(*args: int, a: int, b: int = 4) -> int:
    return str(sum(args) + a + b)


add(4, 3, 4, a=10)

25

### Validate class - `BaseModel`

Kế thừa từ `BaseModel`, chúng ta sử dụng một cơ chế ẩn để thực hiện việc **xác thực dữ liệu**, **phân tích cú pháp**, và **tuần tự hóa**. 

--> Điều này cho phép chúng ta tạo ra các đối tượng phù hợp với các thông số của mô hình.

In [45]:
from pydantic import Field
from pydantic import BaseModel
from typing import Annotated


class Person(BaseModel):
    name: str = Field(min_length=2, max_length=15)

    # cách khác sử dụng Annotated
    age: Annotated[int, Field(default=1, ge=0, le=120)]


# ----

john = Person(name="john", age=20)
print(john)
print("---------------------")

# ----
try:
    mike = Person(name="m", age=0)
except Exception as e:
    print(e)

name='john' age=20
---------------------
2 validation errors for Person
name
  String should have at least 2 characters [type=string_too_short, input_value='m', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/string_too_short
age
  Input should be greater than 0 [type=greater_than, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/greater_than


In [58]:
from pydantic import Field
from pydantic import BaseModel
from pydantic import ConfigDict

from typing import Literal
from enum import Enum


# định nghĩa LLMProviders là dạng string chỉ được nhận 1 trong các giá trị sau : openai, claude.
# Thay vì sử dụng Literal định nghĩa trong attribute
class LLMProviders(str, Enum):
    OPENAI = "openai"
    CLAUDE = "claude"


class LLMParams(BaseModel):
    temperature: int = Field(validation_alias="llm_temperature", ge=0, le=1)
    llm_name: str = Field(
        validation_alias="llm_model_name", serialization_alias="model"
    )


class Payload(BaseModel):
    req_id: str = Field(exclude=True, min_length=2, max_length=15)
    text: str = Field(min_length=5)
    instruction: Literal["embed", "chat"]
    llm_provider: LLMProviders
    llm_params: LLMParams


payload = {
    "req_id": "test",
    "text": "This is a sample text.",
    "instruction": "embed",
    "llm_provider": "openai",
    "misc": "what",
    "llm_params": {"llm_temperature": 0, "llm_model_name": "gpt4o"},
}
validated_payload = Payload(**payload)
print(validated_payload)


req_id='test' text='This is a sample text.' instruction='embed' llm_provider=<LLMProviders.OPENAI: 'openai'> llm_params=LLMParams(temperature=0, llm_name='gpt4o')


In [49]:
validated_payload.model_dump()

{'text': 'This is a sample text.',
 'instruction': 'embed',
 'llm_provider': <LLMProviders.OPENAI: 'openai'>,
 'llm_params': {'temperature': 0, 'llm_name': 'gpt4o'}}

In [50]:
validated_payload.model_dump(by_alias=True)

{'text': 'This is a sample text.',
 'instruction': 'embed',
 'llm_provider': <LLMProviders.OPENAI: 'openai'>,
 'llm_params': {'temperature': 0, 'model': 'gpt4o'}}

Sử dụng `ConfigDict` để sử đổi attribute `model_config` của class, từ đó sửa đổi các tính chất của class:
 - `use_enum_values=True` : khi call model lấy giá trị thì sẽ trả về value của model (`.value`) thay vì để dạng raw là `<LLMProviders.OPENAI: 'openai'>`

In [69]:
# LLMProviders chỉ nhật các giá trị đã được định nghĩa

try:
    LLMProviders("openai---- ver khác")
except Exception as e:
    print(e)

LLMProviders("openai").value

'openai---- ver khác' is not a valid LLMProviders


'openai'

In [56]:
from pydantic import ConfigDict


class Payload(BaseModel):
    req_id: str = Field(exclude=True)
    text: str = Field(min_length=5)
    instruction: Literal["embed", "chat"]
    llm_provider: LLMProviders
    llm_params: LLMParams

    # thêm trường model_config
    model_config = ConfigDict(use_enum_values=True)


validated_payload = Payload(**payload)
validated_payload.model_dump(by_alias=True)

{'text': 'This is a sample text.',
 'instruction': 'embed',
 'llm_provider': 'openai',
 'llm_params': {'temperature': 0, 'model': 'gpt4o'}}

#### `model_validate()`: xác thực dữ liệu từ 1 object

tương tụ như `__init__` method của model (khi tạo đối tượng bằng việc khai báo arguments class đó), thay vào đó là truyền vào `dict` hoặc object

- **Chức năng**: Xác thực (validate) một đối tượng dựa trên mô hình Pydantic đã được định nghĩa. Nó kiểm tra dữ liệu có đúng kiểu và giá trị hay không dựa trên các quy tắc xác thực (validation rules) mà bạn đã định nghĩa trong mô hình.
- **Ứng dụng**: Sử dụng khi bạn muốn đảm bảo rằng dữ liệu đầu vào của mình khớp với các yêu cầu của mô hình trước khi xử lý thêm.

In [72]:
from datetime import datetime
from typing import Optional

from pydantic import BaseModel, ValidationError


class User(BaseModel):
    id: int
    name: str = "John Doe"
    signup_ts: Optional[datetime] = None


u = User.model_validate(
    {"id": 123, "name": "James"}
)  # --> return object of class
u

User(id=123, name='James', signup_ts=None)

- `model_validate_json()`: this validates the provided data as a JSON string or bytes object. (nếu data đang dưới dạng `JSON` hoặc `bytes` thì cách này sẽ nhanh hơn)

#### `model_construct()`: nhận dữ liệu bỏ qua validation

This can be useful in at least a few cases:
- data phức tạp được truyền vào nhưng đã biết được datatype, nên ko cần validate để đảm bảo performance
- khi đã có 1 số hàm validate phía trong và không cần trigger validate lại lần nữa

In [73]:
User.model_construct({"id": "ạkshdajs", "name": [1231231]})

User(name='John Doe', signup_ts=None)

#### `model_dump()`: convert to dict

- **Chức năng**: Trả về một từ điển chứa các trường (fields) và giá trị (values) tương ứng của mô hình.
- **Ứng dụng**: Khi bạn muốn tuần tự hóa (serialize) mô hình thành một dạng dễ xử lý hơn, ví dụ như trước khi lưu vào cơ sở dữ liệu hoặc trả về qua API.

In [74]:
u.model_dump()

{'id': 123, 'name': 'James', 'signup_ts': None}

- `model_dump_json()`: Trả về chuỗi JSON đại diện cho dữ liệu của mô hình (thực chất là kết quả của model_dump() nhưng dưới dạng JSON)

In [75]:
u.model_dump_json()

'{"id":123,"name":"James","signup_ts":null}'

#### `model_copy()` : copy model
- **Chức năng**: Trả về một bản sao của mô hình. 
  - Mặc định là `shallow copy` (bản sao nông, nghĩa là chỉ sao chép đối tượng gốc mà không sao chép sâu các thành phần con bên trong). 
  - Có thể chỉ định để tạo `deep copy` (bản sao sâu).
- **Ứng dụng**: Khi bạn cần một bản sao của mô hình để thực hiện các thay đổi mà không làm ảnh hưởng đến bản gốc.

#### `model_json_schema()` : trả về dictionary of class
- **Chức năng**: Trả về một từ điển có thể chuyển đổi thành JSON, đại diện cho lược đồ JSON (JSON Schema) của mô hình. JSON Schema cung cấp thông tin chi tiết về kiểu dữ liệu, các quy tắc xác thực, v.v.
- **Ứng dụng**: Khi bạn cần cung cấp tài liệu cho API hoặc các dịch vụ khác về cấu trúc dữ liệu được mong đợi.

In [76]:
u.model_json_schema()

{'properties': {'id': {'title': 'Id', 'type': 'integer'},
  'name': {'default': 'John Doe', 'title': 'Name', 'type': 'string'},
  'signup_ts': {'anyOf': [{'format': 'date-time', 'type': 'string'},
    {'type': 'null'}],
   'default': None,
   'title': 'Signup Ts'}},
 'required': ['id'],
 'title': 'User',
 'type': 'object'}

#### `model_post_init()` : chạy hàm sau khi `__init__`
- **Chức năng**: Phương thức này thực hiện các hành động bổ sung sau khi mô hình đã được khởi tạo (sau khi quá trình khởi tạo và xác thực hoàn tất).
- **Ứng dụng**: Sử dụng để thực hiện các thao tác tùy chỉnh sau khi mô hình đã được khởi tạo, như tính toán thêm các giá trị dựa trên các trường đã xác định.

In [89]:
class User(BaseModel):
    id: int
    name: str = "John Doe"
    signup_ts: Optional[datetime] = None

    def model_post_init(self, *args, **kwargs):
        # Tính toán giá trị tổng sau khi mô hình đã được khởi tạo
        print("AAAAAAAAA")

    @computed_field
    @property
    def name_id(self) -> str:
        return f"{self.name}-{self.id}"


u = User(id=123, name="James")

AAAAAAAAA


#### `model_fields` : trả về danh sách định nghĩa các field
- **Chức năng**: Một ánh xạ (mapping) giữa tên trường (field names) và **các định nghĩa của chúng (FieldInfo instances)** (**not Value**). Mỗi trường trong mô hình đều có một định nghĩa riêng về kiểu dữ liệu và các quy tắc xác thực.
- **Ứng dụng**: Sử dụng khi bạn cần truy cập hoặc kiểm tra thông tin về các trường trong mô hình (như kiểu dữ liệu, giá trị mặc định, v.v.).

Chú ý:
- trả về là định nghĩa field, chứ ko phải value
- không bao gồm các `computed_fields` / `property`

In [90]:
u.model_fields

{'id': FieldInfo(annotation=int, required=True),
 'name': FieldInfo(annotation=str, required=False, default='John Doe'),
 'signup_ts': FieldInfo(annotation=Union[datetime, NoneType], required=False, default=None)}

#### `model_computed_fields` : trả về danh sách định nghĩa các field được tính toán
- **Chức năng**: Ánh xạ giữa tên các trường tính toán (computed field names) và các định nghĩa của chúng (ComputedFieldInfo instances). Các trường này không được cung cấp trực tiếp mà được tính toán dựa trên các trường khác.
- **Ứng dụng**: Khi bạn muốn kiểm tra hoặc làm việc với các trường tính toán trong mô hình.

In [91]:
u.model_computed_fields

{'name_id': ComputedFieldInfo(wrapped_property=<property object at 0x000001AFEC920CC0>, return_type=<class 'str'>, alias=None, alias_priority=None, title=None, description=None, deprecated=None, examples=None, json_schema_extra=None, repr=True)}

#### `model_fields_set` : kiểm trả giá trị của trường được khia báo hay mặc định
- **Chức năng**: Tập hợp các trường (fields) đã được cung cấp rõ ràng khi mô hình được khởi tạo, có nghĩa là danh sách các trường đã được truyền input vào thay vì lấy giá trị mặc định
- **Ứng dụng**: Dùng để kiểm tra xem trường nào đã được gán giá trị ban đầu và trường nào có giá trị mặc định.

In [94]:
u.model_fields_set

{'id', 'name'}

## Type of validators

#### Annotated Validators

Có 4 loại validator: `Before`, `After`, `Wrap`, và `Plain`, mỗi loại có vai trò và cách hoạt động khác nhau trong quy trình kiểm tra dữ liệu. 

##### Before Validators:
Chạy trước khi Pydantic thực hiện việc parsing và xác thực dữ liệu. Điều này có nghĩa là các Before Validators xử lý dữ liệu thô đầu vào, trước khi bất kỳ kiểm tra hoặc chuyển đổi nào khác của Pydantic được thực hiện.

- **Nên sử dụng khi**: Khi bạn cần chuyển đổi dữ liệu đầu vào trước khi xác thực, ví dụ: chuyển chuỗi thành số, hoặc làm sạch dữ liệu thô (xóa khoảng trắng, loại bỏ ký tự không cần thiết).
    > chuỗi "123" được chuyển thành số nguyên 123 trước khi Pydantic kiểm tra kiểu dữ liệu và xác thực đầu vào.
- **Thứ tự chạy**: Trong Annotated, `BeforeValidator` chạy đầu tiên (trước cả xác thực `datatype`, `BeforeValidator` và `Field`) , Nếu xét giữa các `BeforeValidator` với nhau thì thứ tự chạy theo chiều từ phải sang trái, hướng về phía ***datatype***.

In [27]:
# Ví dụ: Nếu bạn nhận đầu vào là str và muốn chuyển bỏ chữ cái đầu, sau đó biến nó thành int, nếu là dạng float thì cast float thành int, bạn có thể dùng Before Validator để xử lý trước khi Pydantic tự động làm điều đó.
from typing import Any
from pydantic import Field, BaseModel, BeforeValidator, AfterValidator
from typing import Annotated


def str_to_float(v: Any) -> float:
    print("str_to_float")
    """
    Bỏ ký tự đầu tiên và chuyển thành float
    """
    v_rm0ind = v[1:]
    return float(v_rm0ind)


def float_to_int_to_str(v: float) -> int:
    print("float_to_int_to_str: chuyển về int sau đó chuyển về string")
    assert v > 0, "Value must be positive"
    return str(int(v))


def double(v: Any) -> Any:
    print("double: chuyển về int sau đó nhân đôi")
    return int("1" + v) * 2


def check_squares(v: int) -> int:
    print("check_squares: kiểm tra xem có phải số chính phương")
    assert v**0.5 % 1 == 0, f"{v} is not a square number"
    return v


MyNumber = Annotated[
    str,
    BeforeValidator(float_to_int_to_str),
    BeforeValidator(str_to_float),
    AfterValidator(double),
    AfterValidator(check_squares),
    Field(
        gt=30,
        lt=40,
    ),
]


class MyModel(BaseModel):
    number: MyNumber


MyModel(number="@8.2981")


str_to_float
float_to_int_to_str: chuyển về int sau đó chuyển về string
double: chuyển về int sau đó nhân đôi
check_squares: kiểm tra xem có phải số chính phương


MyModel(number=36)

##### After Validators:
After Validators chạy sau khi Pydantic đã hoàn tất việc parsing và xử lý dữ liệu và xác thực `datatype`, nhưng trước khi kiểm tra `datavalue`. Điều này có nghĩa là bạn sẽ làm việc với dữ liệu đã được xác thực và chuyển đổi thành đúng kiểu dữ liệu mong muốn.

- **Nên sử dụng khi**: Khi bạn chỉ muốn kiểm tra hoặc xác thực dữ liệu sau khi đã qua quá trình chuyển đổi và xác thực của Pydantic. Ví dụ: kiểm tra xem giá trị đã chuyển đổi có hợp lệ hay không.
    > chuỗi "123" được chuyển thành số nguyên 123 trước khi Pydantic kiểm tra kiểu dữ liệu và xác thực đầu vào.
- **Thứ tự chạy**: Trong Annotated, `AfterValidator` chạy sau khi xác thực datatype nhưng sau khi xác thực datavalue (sau `BeforeValidator` và trước `Field`) , Nếu xét giữa các `AfterValidator` với nhau thì thứ tự chạy theo chiều từ trái sang phải, hướng về phía đi khỏi ***datatype*** 

##### Plain Validators:
**Plain Validators** tương tự **Before Validators**, nhưng chúng ngừng quá trình xác thực ngay lập tức. Sử dụng với `Annotated`, chúng đảm bảo rằng nếu điều kiện không thỏa mãn, quá trình xác thực dừng ngay lập tức.

- **Nên sử dụng khi**: Khi bạn cần dừng xác thực ngay lập tức nếu giá trị không thỏa mãn điều kiện cơ bản, như trường hợp số âm ở trên.
    > chuỗi "123" được chuyển thành số nguyên 123 trước khi Pydantic kiểm tra kiểu dữ liệu và xác thực đầu vào.
- **Thứ tự chạy**: Tương đương với **Before Validators**